In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DATAPATH = "data/"

# Check the timeseries data and collect the patient ID 

In [3]:
lvl2_train_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_train.pkl"))
lvl2_dev_imputer = pd.read_pickle(os.path.join(DATAPATH, "lvl2_imputer_dev.pkl"))
lvl2_test_imputer = pd.read_pickle(os.path.join(DATAPATH,"lvl2_imputer_test.pkl"))
Ys = pd.read_pickle(os.path.join(DATAPATH, "Ys.pkl"))

In [4]:
print("Shape of train, dev, test {}, {}, {}.".format((lvl2_train_imputer.shape), (lvl2_dev_imputer.shape), (lvl2_test_imputer.shape)))
print( 402120 / 24, 57432 / 24, 114936 / 24)

Shape of train, dev, test (402240, 312), (57456, 312), (114960, 312).
16755.0 2393.0 4789.0


In [5]:
patient_ids = [] # store all patient ids 
for each_entry in Ys.index:
    patient_ids.append(each_entry[0])

In [6]:
print("Number of total patient {}".format(len(patient_ids)))

Number of total patient 23944


# Select Clinical Notes

In [7]:
admission_df = pd.read_csv(os.path.join(DATAPATH, "ADMISSIONS.csv"))
noteevents_df = pd.read_csv(os.path.join(DATAPATH, "NOTEEVENTS.csv"))
icustays_df = pd.read_csv(os.path.join(DATAPATH, "ICUSTAYS.csv"))

/tmp/ipykernel_1220/4265364739.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents_df = pd.read_csv(os.path.join(DATAPATH, "NOTEEVENTS.csv"))


In [8]:
noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count'])

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,DESCRIPTION,CGID,ISERROR,TEXT
,count,count,count,count,count,count,count,count,count,count
CATEGORY,,,,,,,,,,
Case Management,967,967,954,967,967,967,967,967,14,967
Consult,98,98,98,98,98,98,98,98,0,98
Discharge summary,59652,59652,59652,59652,0,0,59652,0,0,59652
ECG,209051,209051,138190,209051,0,0,209051,0,0,209051
Echo,45794,45794,34037,45794,0,0,45794,0,0,45794
General,8301,8301,8209,8301,8260,8301,8301,8301,65,8301
Nursing,223556,223556,220758,223556,222172,223556,223556,223556,374,223556
Nursing/other,822497,822497,821258,822497,822497,822497,822497,822497,0,822497


In [9]:
note_categories = noteevents_df.groupby(noteevents_df.CATEGORY).agg(['count']).index

In [10]:
#selected_note_types = ['Nursing/other', 'Radiology', 'Nursing', 'ECG', 'Physician', 'Echo', 'Respiratory', 'Nutrition']
selected_note_types = []
for each_cat in list(note_categories):
    if each_cat != 'Discharge summary':
        selected_note_types.append(each_cat)

In [12]:
noteevents_df.shape

(2083180, 11)

### Select based on note category

In [13]:
sub_notes = noteevents_df[noteevents_df.CATEGORY.isin(selected_note_types)]

In [14]:
sub_notes.shape

(2023528, 11)

### Drop no chart notes

In [15]:
missing_chardate_index = []
for each_note in sub_notes.itertuples():
    if isinstance(each_note.CHARTTIME, str):
        continue
    if np.isnan(each_note.CHARTTIME):
        missing_chardate_index.append(each_note.Index)
print ("{} of notes does not charttime.".format(len(missing_chardate_index)))
print(sub_notes.shape)

256914 of notes does not charttime.
(2023528, 11)


In [16]:
sub_notes.drop(missing_chardate_index, inplace=True )
print(sub_notes.shape)

(1766614, 11)


/tmp/ipykernel_1220/3231713327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_notes.drop(missing_chardate_index, inplace=True )


### Select based on patient id

In [17]:
sub_notes = sub_notes[sub_notes.SUBJECT_ID.isin(patient_ids)] ## select based on patient id

In [18]:
sub_notes.shape

(745974, 11)

### Select based on time limit (24 hours) 

In [20]:
MIMIC_EXTRACT_DATA = "data/all_hourly_data.h5"
stats = pd.read_hdf(MIMIC_EXTRACT_DATA, 'patients')
TIMELIMIT = 1 # 1day

In [21]:
stats.shape

(34472, 28)

In [22]:
stats.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [23]:
new_stats = stats.reset_index()

In [24]:
new_stats.rename(columns = {"subject_id": "SUBJECT_ID", "hadm_id": "HADM_ID"}, inplace=True)

In [25]:
print(new_stats.shape, sub_notes.shape)

(34472, 31) (745974, 11)


In [26]:
df_adm_notes = pd.merge(sub_notes[['ROW_ID','SUBJECT_ID','HADM_ID','CHARTTIME', 'CATEGORY', 'TEXT']],
                        new_stats[['SUBJECT_ID','HADM_ID','icustay_id','age','admittime','dischtime', 'deathtime', 
                                  'intime', 'outtime', 'los_icu', 'mort_icu', 'mort_hosp', 'hospital_expire_flag',
                                  'hospstay_seq', 'max_hours']], 
                        on = ['SUBJECT_ID'],
                        how = 'left')

In [27]:
df_adm_notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID_x,CHARTTIME,CATEGORY,TEXT,HADM_ID_y,icustay_id,age,admittime,dischtime,deathtime,intime,outtime,los_icu,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours
0,316234,29075,179159.0,2116-02-07 14:08:00,Nursing,"67M w/ h/o multiplemyeloma Dx [**2111**], neur...",179159,272120,67.798859,2116-02-05 23:46:00,2116-02-17 17:00:00,NaT,2116-02-05 23:46:48,2116-02-07 23:33:32,1.990787,0,0,0,1,47
1,316235,18082,181163.0,2156-03-12 14:23:00,Nursing,"[**Age over 90 52**]F with COPD on home O2, CA...",179564,231140,300.005452,2154-11-14 22:49:00,2154-11-24 13:00:00,NaT,2154-11-15 14:11:19,2154-11-20 22:23:57,5.342106,0,0,0,1,128
2,316236,18082,181163.0,2156-03-12 14:28:00,Nursing,"[**Age over 90 52**]F with COPD on home O2, CA...",179564,231140,300.005452,2154-11-14 22:49:00,2154-11-24 13:00:00,NaT,2154-11-15 14:11:19,2154-11-20 22:23:57,5.342106,0,0,0,1,128
3,316238,29075,179159.0,2116-02-07 15:37:00,Physician,Chief Complaint:\n 24 Hour Events:\n EGD d...,179159,272120,67.798859,2116-02-05 23:46:00,2116-02-17 17:00:00,NaT,2116-02-05 23:46:48,2116-02-07 23:33:32,1.990787,0,0,0,1,47
4,316239,18839,136227.0,2193-02-08 15:39:00,Rehab Services,Attending Physician: [**Name10 (NameIs) 116**]...,152908,295649,82.419243,2190-05-15 13:56:00,2190-05-25 19:28:00,NaT,2190-05-15 23:15:47,2190-05-19 15:02:25,3.657384,0,0,0,1,87


In [28]:
df_adm_notes['CHARTTIME'] = pd.to_datetime(df_adm_notes['CHARTTIME'])

In [29]:
df_less_n = df_adm_notes[((df_adm_notes['CHARTTIME']-df_adm_notes['intime']).dt.total_seconds()/(24*60*60))<TIMELIMIT]

In [30]:
df_less_n.shape

(181483, 20)

# Save clinical notes

In [31]:
pd.to_pickle(df_less_n, "data/sub_notes.p")